In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import os

In [2]:
geolocator = Nominatim(user_agent="measurements")

In [17]:
locs = ["Plank Town Brewing Company", "PublicHouse", "Willamalane Adult Activity Center", "Dorris Ranch", "Heron Playground",
        "University of Oregon", "999 Willamette St, Eugene, OR 97401", "South Eugene High School", "1061 W 2nd Ave, Eugene, OR 97402",
        "101 S A St, Springfield, OR 97477", "Willamalane Park Swim Center", "Bob Keefer Center", "The Shoppes at Gateway",
        "1025 G St, Springfield, OR 97477", "30 E St, Springfield, OR 97477", "625 West Centennial Blvd", 
        "3377 Riverbend Dr, Springfield, OR 97477"]

In [4]:
def getCoordinates(place):
    location = geolocator.geocode(place)
    address, (latitude, longitude) = location
    return [place, longitude, latitude] 

In [16]:
getCoordinates("999 Willamette St, Eugene, OR 97401")

['999 Willamette St, Eugene, OR 97401', -123.0923105069869, 44.0490291]

In [8]:
data = []

In [6]:
names = ['Plank Town', 'Public House', 'Willamalane Adult Activity Center', 'Dorris Ranch', 'Heron Park',
        'University of Oregon', 'Downtown Eugene', 'South Eugene', 'Whiteaker', 'Springfield Chamber of Commerce', 
         'Willamalane Park Swim Center', 'Bob Keefer Center', 'Gateway Mall', 'Catholic Community Services',
         'Bungalow Market', 'Centennial Center', 'PeaceHealth Riverbend']

In [18]:
for loc in locs:
    print(loc)
    ls = getCoordinates(loc)
    ls.insert(0, names[locs.index(loc)])
    data.append(ls)

Plank Town Brewing Company
PublicHouse
Willamalane Adult Activity Center
Dorris Ranch
Heron Playground
University of Oregon
999 Willamette St, Eugene, OR 97401
South Eugene High School
1061 W 2nd Ave, Eugene, OR 97402
101 S A St, Springfield, OR 97477
Willamalane Park Swim Center
Bob Keefer Center
The Shoppes at Gateway
1025 G St, Springfield, OR 97477
30 E St, Springfield, OR 97477
625 West Centennial Blvd
3377 Riverbend Dr, Springfield, OR 97477


In [20]:
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Name', 'Address', 'Longitude', 'Latitude'])

In [21]:
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [23]:
outpath = r'T:\DCProjects\Support\SPR\data'

In [24]:
selected_facilities_o = gpd.read_file(r"T:\DCProjects\Support\SPR\BikeShare\BikeShare.gdb", layer="selected_facilities")

In [25]:
selected_facilities_o.columns

Index(['label', 'label_full', 'label_add', 'type', 'type_full', 'eug_amenity',
       'school_district', 'address_geofeat_id', 'address_uuid', 'x_coordinate',
       'y_coordinate', 'longitude', 'latitude', 'facility_geo_id', 'geometry'],
      dtype='object')

In [26]:
selected_facilities_o.type_full.unique()

array(['LTD Transit Sta', 'Park', 'Private School', 'High School',
       'Middle School', 'Elementary Sch', 'City Hall'], dtype=object)

In [27]:
selected_facilities_s1 = selected_facilities_o[(selected_facilities_o.type_full.isin(['LTD Transit Sta','City Hall']))]

In [28]:
parks = ['Island Park', 'Dorris Ranch', 'Millrace Park', 'Willamalane Park', 
         'Meadow Park', 'Heron Park', 'Alton Baker Park', 
         'Amazon Park','Skinner Butte Park', 'Monroe Park']

In [29]:
selected_parks = [park for park in parks if park in selected_facilities_o.label_full.values]

In [30]:
selected_facilities_s2 = selected_facilities_o[selected_facilities_o.label_full.isin(selected_parks)]

In [31]:
selected_facilities_s = selected_facilities_s1.append(selected_facilities_s2)

In [32]:
selected_facilities = selected_facilities_s[['label_full', 'longitude', 'latitude', 'geometry']]

In [33]:
selected_facilities.rename(columns ={'label_full':'Name', 'longitude':'Longitude', 'latitude':'Latitude'}, inplace=True)

C:\ProgramData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
selected_facilities = selected_facilities.append(gdf[['Name', 'Longitude', 'Latitude', 'geometry']])

In [35]:
selected_facilities.to_file(os.path.join(outpath, 'points_of_interest.shp'))